# data_prep
Time to mess with this grid again! It's been a while :)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import sklearn
from sklearn.decomposition import PCA

plt.style.use("Solarize_Light2")
plt.rcParams.update({"axes.edgecolor": "black"})
plt.rcParams.update({"text.color": "black"})
plt.rcParams.update({"axes.labelcolor": "black"})
plt.rcParams.update({"xtick.color": "black"})
plt.rcParams.update({"ytick.color": "black"})
plt.rcParams.update({"font.family": "monospace"})

## load in data

In [2]:
df_full = pd.read_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob.h5", key='df')
df_full.describe()

,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,radius,luminosity,effective_T,star_feh,nu_0_4,...,log_nu_0_32_std,log_nu_0_33_std,log_nu_0_34_std,log_nu_0_35_std,log_nu_0_36_std,log_nu_0_37_std,log_nu_0_38_std,log_nu_0_39_std,log_nu_0_40_std,log_calc_effective_T_std
count,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,...,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06,2.448681e+06
mean,1.021885e+00,1.440698e-02,2.814693e-01,2.117984e+00,5.000177e+00,1.312728e+00,2.116482e+00,5.870890e+03,-1.127145e-01,5.459277e+02,...,3.629539e-15,2.700240e-15,1.435547e-15,-4.423639e-15,8.003407e-15,-8.417543e-15,1.289578e-15,-3.185318e-15,3.670395e-15,-4.006495e-14
std,1.175610e-01,9.677123e-03,2.805593e-02,2.879175e-01,3.405371e+00,4.813547e-01,1.572419e+00,5.674285e+02,3.499532e-01,2.021373e+02,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,8.000000e-01,3.869061e-03,2.400000e-01,1.700000e+00,2.966411e-02,6.990236e-01,1.321735e-01,4.099787e+03,-1.077903e+00,1.545097e+02,...,-2.599363e+00,-2.594960e+00,-2.591708e+00,-2.589646e+00,-2.588901e+00,-2.589287e+00,-2.590747e+00,-2.593002e+00,-2.595848e+00,-3.657017e+00
25%,9.200000e-01,6.471429e-03,2.600000e-01,1.900000e+00,2.309880e+00,9.719693e-01,9.371542e-01,5.495672e+03,-4.000000e-01,3.986516e+02,...,-5.643108e-01,-5.676179e-01,-5.705508e-01,-5.723483e-01,-5.730070e-01,-5.725672e-01,-5.713737e-01,-5.700038e-01,-5.685747e-01,-6.332417e-01
50%,1.040000e+00,1.077191e-02,2.800000e-01,2.100000e+00,4.275231e+00,1.160016e+00,1.698469e+00,5.863863e+03,-1.000000e-01,5.591040e+02,...,2.417789e-01,2.415099e-01,2.405877e-01,2.389688e-01,2.369176e-01,2.348740e-01,2.333131e-01,2.326247e-01,2.327753e-01,3.593147e-02
75%,1.120000e+00,2.007835e-02,3.000000e-01,2.300000e+00,7.159688e+00,1.498369e+00,2.883064e+00,6.223793e+03,1.837200e-01,6.947397e+02,...,7.569644e-01,7.578097e-01,7.587334e-01,7.595544e-01,7.599729e-01,7.598420e-01,7.591873e-01,7.580180e-01,7.566985e-01,6.506511e-01
max,1.200000e+00,3.897971e-02,3.200000e-01,2.500000e+00,1.399997e+01,2.873431e+00,1.163902e+01,7.998610e+03,5.094573e-01,1.021990e+03,...,1.617024e+00,1.618137e+00,1.620054e+00,1.622624e+00,1.625764e+00,1.629202e+00,1.632519e+00,1.633559e+00,1.635647e+00,3.239603e+00


## define inputs and outputs

In [4]:
#### define inputs
inputs = ['log_initial_mass_std', 'log_initial_Zinit_std', 'log_initial_Yinit_std', 'log_initial_MLT_std', 'log_star_age_std']

#### define outputs
classical_outputs = ['log_radius_std', 'log_luminosity_std', 'star_feh_std']
astero_outputs = [f'log_nu_0_{i+1}_std' for i in range(5,40)]

outputs = classical_outputs+astero_outputs

## make train/test split

In [5]:
df_train_val, df_test = sklearn.model_selection.train_test_split(df_full, test_size=0.05, random_state=42) #<- DO NOT CHANGE!
df_train, df_val = sklearn.model_selection.train_test_split(df_train_val, test_size=0.05/0.95, random_state=74)

print("Training set: ", len(df_train))
print("Validation set: ", len(df_val))
print("Test set: ", len(df_test))

Training set:  2203811
Validation set:  122435
Test set:  122435


In [6]:
df_train.to_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob_train_fp.h5", key="df")
df_val.to_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob_val_fp.h5", key="df")
df_test.to_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob_test_fp.h5", key="df")

In [7]:
df_train[inputs+outputs].to_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob_train_slim_fp.h5", key="df")
df_val[inputs+outputs].to_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob_val_slim_fp.h5", key="df")
df_test[inputs+outputs].to_hdf("/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/bob_test_slim_fp.h5", key="df")

# check train_fp, val_fp != train_pf, val_pf

In [8]:
train_fp = df_train
val_fp = df_val

train_pf = pd.read_hdf("/home/oxs235/datastorage/repos_data/ojscutt/pitchfork-bb/data/bob_train.h5", key='df')
val_pf = pd.read_hdf("/home/oxs235/datastorage/repos_data/ojscutt/pitchfork-bb/data/bob_val.h5", key='df')

In [16]:
train_fp.describe() == train_pf.describe()

,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,radius,luminosity,effective_T,star_feh,nu_0_4,...,log_nu_0_32_std,log_nu_0_33_std,log_nu_0_34_std,log_nu_0_35_std,log_nu_0_36_std,log_nu_0_37_std,log_nu_0_38_std,log_nu_0_39_std,log_nu_0_40_std,log_calc_effective_T_std
count,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
mean,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
std,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
min,True,True,True,True,False,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
25%,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50%,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
75%,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
max,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [17]:
val_fp.describe() == val_pf.describe()

,initial_mass,initial_Zinit,initial_Yinit,initial_MLT,star_age,radius,luminosity,effective_T,star_feh,nu_0_4,...,log_nu_0_32_std,log_nu_0_33_std,log_nu_0_34_std,log_nu_0_35_std,log_nu_0_36_std,log_nu_0_37_std,log_nu_0_38_std,log_nu_0_39_std,log_nu_0_40_std,log_calc_effective_T_std
count,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
mean,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
std,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
min,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
25%,True,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
50%,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
75%,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
max,True,True,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


## evaluate pca and save

In [18]:
"""
PCA
"""
#### define pca global vars
n_components = 15
seed = 42

#### define and fit pca
pca = PCA(n_components=n_components, random_state=seed)
pca.fit(df_train[astero_outputs])

np.savetxt('/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/pca_mean_fp.csv', pca.mean_)
np.savetxt('/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/pca_components_fp.csv', pca.components_)

#### print variance with chosen n_comps
print("Explained variance ratio with n_comps = " + str(n_components) + " is " + str(sum(pca.explained_variance_ratio_)))

Explained variance ratio with n_comps = 15 is 0.9999999417574811


In [19]:
pca.components_

array([[ 1.69021451e-01,  1.69018740e-01,  1.69017290e-01,
         1.69019232e-01,  1.69023085e-01,  1.69027240e-01,
         1.69030402e-01,  1.69032405e-01,  1.69033702e-01,
         1.69034667e-01,  1.69035349e-01,  1.69035775e-01,
         1.69036039e-01,  1.69036194e-01,  1.69036237e-01,
         1.69036182e-01,  1.69036051e-01,  1.69035754e-01,
         1.69035284e-01,  1.69034385e-01,  1.69032939e-01,
         1.69031228e-01,  1.69029838e-01,  1.69029089e-01,
         1.69028987e-01,  1.69029350e-01,  1.69029968e-01,
         1.69030644e-01,  1.69031233e-01,  1.69031623e-01,
         1.69031835e-01,  1.69031906e-01,  1.69031896e-01,
         1.69031895e-01,  1.69031885e-01],
       [ 2.01112698e-01,  3.22999317e-01,  3.59608169e-01,
         3.44615805e-01,  2.97259939e-01,  2.39654678e-01,
         1.91972876e-01,  1.57383763e-01,  1.28176906e-01,
         9.92575712e-02,  7.16614920e-02,  4.81506470e-02,
         2.90085326e-02,  1.23436362e-02, -3.54845431e-03,
        -1.92

## define weights for WMSE

In [20]:
log_weights = (1/np.log(10)) * np.array(([0.01, 0.02, 0.0001]+[0.1 for i in range(len(astero_outputs))]) / df_train[["radius", "luminosity", "star_feh"] + [f'nu_0_{i+1}' for i in range(5,40)]].mean())

log_weights = log_weights / df_train[["log_radius", "log_luminosity", "star_feh"] + [f'log_nu_0_{i+1}' for i in range(5,40)]].std()

weights = log_weights.values.tolist()

weights[2] = 0.01

np.savetxt('/home/oxs235/datastorage/repos_data/ojscutt/fitchpork-train/data/WMSE_weights_fp.csv', np.array(weights))